In [241]:
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import dbManager  # Assuming dbManager is properly set up for using SQLAlchemy


In [242]:
load_dotenv('db.env')

DB_PASSWORD = os.getenv("DB_PASSWORD")
URI = 'dublinbikes.clw8uqmac8qf.eu-west-1.rds.amazonaws.com'
PORT = 3306
USER = 'admin'
DB = 'dbikes'

# Connect to the db
connection_string = f"mysql+mysqlconnector://{USER}:{DB_PASSWORD}@{URI}:{PORT}/{DB}"
engine = create_engine(connection_string, echo=True)


# Testing
try:
    connection = engine.connect()
    print("Connection established successfully.")

except Exception as e:
    print("Failed to establish connection:", e)


2024-04-03 19:24:39,572 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-04-03 19:24:39,573 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 19:24:39,598 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-04-03 19:24:39,598 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 19:24:39,612 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-04-03 19:24:39,612 INFO sqlalchemy.engine.Engine [raw sql] {}
Connection established successfully.


In [243]:
import pandas as pd

def get_station_data(number):
    query = f"SELECT * FROM availability WHERE number = {number}"
    # Execute the query and fetch results directly into a DataFrame
    df = pd.read_sql(query, engine)
    return df


In [244]:
def get_weather_data():
    query = "SELECT * FROM currentweather"
    df = pd.read_sql(query, engine)
    return df


In [245]:
def create_weather_df():
    weather_df = get_weather_data()
    weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp'], unit='s')
    # Create a merge key that includes up to the minute
    weather_df['merge_key'] = weather_df['timestamp'].dt.strftime('%Y-%m-%d %H:%M')
    return weather_df

def availability_df(station_number):
    availability_df = get_station_data(station_number)
    availability_df['timestamp'] = pd.to_datetime(availability_df['timestamp'], unit='s')
    # Create a merge key that includes up to the minute
    availability_df['merge_key'] = availability_df['timestamp'].dt.strftime('%Y-%m-%d %H:%M')
    return availability_df


def merge_dfs(weather_df, station_number):
    availability = availability_df(station_number)
    # Merge on the new merge_key
    merged_df = pd.merge(weather_df, availability, on='merge_key', how='inner')
    # Optionally, convert merge_key back to datetime for further time-based analysis
    merged_df['timestamp'] = pd.to_datetime(merged_df['merge_key'])
    return merged_df


In [246]:
merge_dfs(create_weather_df(), 2).tail()

2024-04-03 19:24:39,716 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-03 19:24:39,716 INFO sqlalchemy.engine.Engine DESCRIBE `dbikes`.`SELECT * FROM currentweather`
2024-04-03 19:24:39,717 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 19:24:39,739 INFO sqlalchemy.engine.Engine SELECT * FROM currentweather
2024-04-03 19:24:39,739 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 19:24:39,854 INFO sqlalchemy.engine.Engine ROLLBACK
2024-04-03 19:24:39,901 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-03 19:24:39,901 INFO sqlalchemy.engine.Engine DESCRIBE `dbikes`.`SELECT * FROM availability WHERE number = 2`
2024-04-03 19:24:39,901 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 19:24:39,917 INFO sqlalchemy.engine.Engine SELECT * FROM availability WHERE number = 2
2024-04-03 19:24:39,917 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 19:24:40,086 INFO sqlalchemy.engine.Engine ROLLBACK


,id,temperature,description,wind_speed,rainfall,timestamp_x,merge_key,availability_id,number,status,bikes,stands,mechanicalBikes,electricalBikes,electricalInternalBatteryBikes,electricalRemovableBatteryBikes,lastUpdate,timestamp_y,timestamp
10080,10089,14.26,scattered clouds,2.57,0.0,2024-04-03 18:00:04,2024-04-03 18:00,1208370,2,OPEN,4,16,3,1,0,1,2024-04-03 17:55:29,2024-04-03 18:00:04,2024-04-03 18:00:00
10081,10090,14.68,scattered clouds,3.13,0.0,2024-04-03 18:05:02,2024-04-03 18:05,1208484,2,OPEN,3,17,2,1,0,1,2024-04-03 18:01:23,2024-04-03 18:05:02,2024-04-03 18:05:00
10082,10091,14.73,scattered clouds,2.68,0.0,2024-04-03 18:10:03,2024-04-03 18:10,1208598,2,OPEN,4,16,3,1,0,1,2024-04-03 18:09:00,2024-04-03 18:10:02,2024-04-03 18:10:00
10083,10092,14.79,clear sky,3.60,0.0,2024-04-03 18:15:03,2024-04-03 18:15,1208712,2,OPEN,6,14,5,1,0,1,2024-04-03 18:14:14,2024-04-03 18:15:02,2024-04-03 18:15:00
10084,10093,14.73,clear sky,3.60,0.0,2024-04-03 18:20:03,2024-04-03 18:20,1208826,2,OPEN,7,13,6,1,0,1,2024-04-03 18:15:48,2024-04-03 18:20:02,2024-04-03 18:20:00


In [247]:
merged_df = merge_dfs(create_weather_df(), 2)

2024-04-03 19:24:40,144 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-03 19:24:40,145 INFO sqlalchemy.engine.Engine DESCRIBE `dbikes`.`SELECT * FROM currentweather`
2024-04-03 19:24:40,145 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 19:24:40,159 INFO sqlalchemy.engine.Engine SELECT * FROM currentweather
2024-04-03 19:24:40,160 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 19:24:40,268 INFO sqlalchemy.engine.Engine ROLLBACK
2024-04-03 19:24:40,345 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-03 19:24:40,345 INFO sqlalchemy.engine.Engine DESCRIBE `dbikes`.`SELECT * FROM availability WHERE number = 2`
2024-04-03 19:24:40,346 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 19:24:40,374 INFO sqlalchemy.engine.Engine SELECT * FROM availability WHERE number = 2
2024-04-03 19:24:40,375 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 19:24:40,504 INFO sqlalchemy.engine.Engine ROLLBACK


In [248]:
# Feature Engineering: Add day of the week and hour as features
merged_df['day_of_week'] = merged_df['timestamp'].dt.dayofweek
merged_df['hour'] = merged_df['timestamp'].dt.hour

# Select features and target for the model
features = ['temperature', 'rainfall', 'day_of_week', 'hour']  # Example feature set
target = 'bikes'  # Target variable


In [249]:
from sklearn.model_selection import train_test_split

# Assuming no missing values or categorical variables for simplicity
X = merged_df[features]
y = merged_df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
